# Mobility Meeting Scheduler

### Constants

In [29]:
SOLVER_TIME_LIMIT = 2000

MAXIMUM_FLIGHT_COST = 2500
MAXIMUM_USEFUL_TIME = 2628288

### Import Data

#### Students

In [30]:
import json
from datetime import datetime, timedelta

# Import students information
file = open('../../data/students.json')

input = json.load(file)

DESTINATIONS = input['destinations']
MINIMUM_USEFUL_TIME = input['minimumTime']
STUDENTS_INFO = input['students']
N_STUDENTS = len(STUDENTS_INFO)

STUDENTS_ORIGINS = [DESTINATIONS.index(x) for x in [student["city"] for student in STUDENTS_INFO]]
STUDENTS_STOPS = [student["maxConnections"] for student in STUDENTS_INFO]
STUDENTS_DURATIONS = [student["maxDuration"] for student in STUDENTS_INFO]
STUDENTS_DEPARTURES = [int(round(datetime.strptime(student["earliestDeparture"], '%H:%M:%S').timestamp())) for student in STUDENTS_INFO]
STUDENTS_ARRIVALS = [int(round(datetime.strptime(student["latestArrival"], '%H:%M:%S').timestamp())) for student in STUDENTS_INFO]

STUDENS_AVAILABILITIES = []
for student in STUDENTS_INFO:
    studentIntervals = []
    for start, end in student["availability"]:
        studentIntervals.append(int(round(datetime.strptime(start, '%d/%m/%Y').timestamp())))
        studentIntervals.append(int(round(datetime.strptime(end, '%d/%m/%Y').timestamp())))
    STUDENS_AVAILABILITIES.append(studentIntervals)

N_MAX_INTERVALS = max([int(len(x) / 2) for x in STUDENS_AVAILABILITIES])

print("Imported", N_STUDENTS, "students,", len(DESTINATIONS), "destinations and the minimum time.")

Imported 3 students, 7 destinations and the minimum time.


#### Flights

In [31]:
# Import flights
file = open('../../data/flights.json')

FLIGHTS = json.load(file)

FLIGHTS_ORIGINS = [DESTINATIONS.index(x) for x in [flight["origin"] for flight in FLIGHTS]]
FLIGHTS_DESTINATIONS = [DESTINATIONS.index(x) for x in [flight["destination"] for flight in FLIGHTS]]

FLIGHTS_DEPARTURES = [int(round(datetime.strptime(flight["departure"], '%d/%m/%Y, %H:%M:%S').timestamp())) for flight in FLIGHTS]
FLIGHTS_DEPARTURE_TIMES = [int(round(datetime.strptime(datetime.fromtimestamp(departure).time().isoformat(), '%H:%M:%S').timestamp())) for departure in FLIGHTS_DEPARTURES]

FLIGHTS_ARRIVALS = [int(round(datetime.strptime(flight["arrival"], '%d/%m/%Y, %H:%M:%S').timestamp())) for flight in FLIGHTS]
FLIGHTS_ARRIVAL_TIMES = [int(round(datetime.strptime(datetime.fromtimestamp(arrival).time().isoformat(), '%H:%M:%S').timestamp())) for arrival in FLIGHTS_ARRIVALS]

FLIGHTS_DURATIONS = [flight["duration"] for flight in FLIGHTS]
FLIGHTS_COSTS = [int(flight["price"]) for flight in FLIGHTS]
FLIGHTS_STOPS = [flight["stops"] for flight in FLIGHTS]

print("Imported", len(FLIGHTS), "flights!")

Imported 9311 flights!


#### Model

In [32]:
from docplex.cp.model import CpoModel
from docplex.cp.expression import INT_MIN, INT_MAX

model = CpoModel()

#### Variables

In [33]:
# Chosen Destination
Destination = model.integer_var(0, len(DESTINATIONS) - 1, "Destination")

# Indexes of the flights each student has to take
StudentsFlights = [model.integer_var_list(2, 0, len(FLIGHTS) - 1) for i in range(N_STUDENTS)]

# Student avaliability interval
StudentsAvailabilityIntervals = model.integer_var_list(N_STUDENTS, 0, N_MAX_INTERVALS, "Interval")

# Cost for each of the students
StudentsCosts = model.integer_var_list(N_STUDENTS, 0, MAXIMUM_FLIGHT_COST, "StudentCost")

# Total trip cost
TotalCost = model.integer_var(0, MAXIMUM_FLIGHT_COST * N_STUDENTS, "TotalCost")

# Useful time
UsefulTime = model.integer_var(0, MAXIMUM_USEFUL_TIME, "UsefulTime")

# Separated Time
SeparatedTime = model.integer_var(0, MAXIMUM_USEFUL_TIME, "SeparatedTime")

#### Constraints

In [34]:
for i in range(N_STUDENTS):
    Outgoing, Incoming = StudentsFlights[i]
    StudentOrigin = model.element(STUDENTS_ORIGINS, i)

    # Origin of the flights
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ORIGINS, Outgoing) == StudentOrigin))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ORIGINS, Incoming) == Destination))

    # Destination of the flights
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DESTINATIONS, Outgoing) == Destination))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DESTINATIONS, Incoming) == StudentOrigin))

    # Availability
    startAvailability = model.element(STUDENS_AVAILABILITIES[i], StudentsAvailabilityIntervals[i] * 2)
    endAvailability = model.element(STUDENS_AVAILABILITIES[i], StudentsAvailabilityIntervals[i] * 2 + 1)
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DEPARTURES, Outgoing) >= startAvailability))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ARRIVALS, Incoming) <= endAvailability))

    # Outgoing arrival time must be before Incoming departure time
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ARRIVALS, Outgoing) < model.element(FLIGHTS_DEPARTURES, Incoming)))

    # Earliest departure
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DEPARTURE_TIMES, Outgoing) >= model.element(STUDENTS_DEPARTURES, i)))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DEPARTURE_TIMES, Incoming) >= model.element(STUDENTS_DEPARTURES, i)))

    # Latest arrival
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ARRIVAL_TIMES, Outgoing) <= model.element(STUDENTS_ARRIVALS, i)))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ARRIVAL_TIMES, Incoming) <= model.element(STUDENTS_ARRIVALS, i)))

    # Maximum number of stops
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_STOPS, Outgoing) <= model.element(STUDENTS_STOPS, i)))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_STOPS, Incoming) <= model.element(STUDENTS_STOPS, i)))

    # Maximum flight duration
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DURATIONS, Outgoing) <= model.element(STUDENTS_DURATIONS, i)))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DURATIONS, Incoming) <= model.element(STUDENTS_DURATIONS, i)))

    # Student Cost
    studentCost = model.element(FLIGHTS_COSTS, StudentsFlights[i][0]) + model.element(FLIGHTS_COSTS, StudentsFlights[i][1])
    model.add(model.conditional(StudentOrigin == Destination, 0, studentCost) == StudentsCosts[i])


firstOutgoingTime = model.min([model.conditional(model.element(STUDENTS_ORIGINS, i) != Destination, model.element(FLIGHTS_ARRIVALS, StudentsFlights[i][0]), INT_MAX) for i in range(N_STUDENTS)])
lastOutgoingTime = model.max([model.conditional(model.element(STUDENTS_ORIGINS, i) != Destination, model.element(FLIGHTS_ARRIVALS, StudentsFlights[i][0]), INT_MIN) for i in range(N_STUDENTS)])
firstIncomingTime = model.min([model.conditional(model.element(STUDENTS_ORIGINS, i) != Destination, model.element(FLIGHTS_DEPARTURES, StudentsFlights[i][1]), INT_MAX) for i in range(N_STUDENTS)])
lastIncomingTime = model.max([model.conditional(model.element(STUDENTS_ORIGINS, i) != Destination, model.element(FLIGHTS_DEPARTURES, StudentsFlights[i][1]), INT_MIN) for i in range(N_STUDENTS)])

# Useful Time
model.add(UsefulTime == firstIncomingTime - lastOutgoingTime)
model.add(UsefulTime >= MINIMUM_USEFUL_TIME)

# Separated Time
model.add(SeparatedTime == (lastOutgoingTime - firstOutgoingTime) + (lastIncomingTime - firstIncomingTime))

# Total cost
model.add(TotalCost == model.sum(StudentsCosts))

# Minimize function
model.add(model.minimize(TotalCost / UsefulTime))

#### Solve

In [35]:
print("Solving model....")

solution = model.solve(TimeLimit = SOLVER_TIME_LIMIT)

Solving model....
 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 16 variables, 52 constraints
 ! TimeLimit            = 2000
 ! Initial process time : 0.53s (0.53s extraction + 0.00s propagation)
 !  . Log search space  : 173.4 (before), 173.4 (after)
 !  . Memory usage      : 9.3 MB (before), 9.3 MB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         16                 -
 + New bound is 0
                        0         16    1   F        -
 + New bound is 0.000002282425
 *     0.4620833       10  3.76s        1      (gap is 100.00%)
 *   0.008369829       31  3.76s        1      (gap is 99.97%)
 *   0.007006410      313  3.76s        3      (gap is 99.97%)
 *   0.006732154      970  3.76s        3      (gap is 99.97%)
     0.006732154     1000         

#### Solution

In [38]:
def print_flight(flight):
    print("         ", flight["departure"], "->", flight["arrival"], "(", timedelta(minutes = flight["duration"]), ")")
    print("          Flight costs " + flight["price"] + "€ with " + str(flight["stops"]) + " stops.")

if solution:
    # solution.print_solution()
    print("Solution status: " + solution.get_solve_status())
    print("--> Destination: " + DESTINATIONS[solution[Destination]])
    print("--> Total Cost: " + str(solution[TotalCost]) + "€")
    print("--> Useful Time:", solution[UsefulTime], "(", timedelta(seconds = solution[UsefulTime]), ")")
    print("--> Separated Time:", solution[SeparatedTime], "(", timedelta(seconds = solution[SeparatedTime]), ")")
    print("--> Students:")

    for i in range(N_STUDENTS):
        Outgoing, Incoming = StudentsFlights[i]

        print("    --> Student " + str(i + 1) + " departing from " + STUDENTS_INFO[i]["city"] + ":")

        if STUDENTS_ORIGINS[i] == solution[Destination]:
            print("       - Student need not to take any flights!")
        else:
            print("        OUTGOING FLIGHT:")
            print_flight(FLIGHTS[solution[Outgoing]])
            print("        INCOMING FLIGHT:")
            print_flight(FLIGHTS[solution[Incoming]])

else:
    print("No solution found")

Solution status: Optimal
--> Destination: milano
--> Total Cost: 64€
--> Useful Time: 1125900 ( 13 days, 0:45:00 )
--> Separated Time: 116400 ( 1 day, 8:20:00 )
--> Students:
    --> Student 1 departing from budapest:
        OUTGOING FLIGHT:
          01/06/2022, 08:15:00 -> 01/06/2022, 09:50:00 ( 1:35:00 )
          Flight costs 13€ with 0 stops.
        INCOMING FLIGHT:
          14/06/2022, 21:00:00 -> 14/06/2022, 22:30:00 ( 1:30:00 )
          Flight costs 5€ with 0 stops.
    --> Student 2 departing from zagreb:
        OUTGOING FLIGHT:
          01/06/2022, 19:00:00 -> 01/06/2022, 20:15:00 ( 1:15:00 )
          Flight costs 10€ with 0 stops.
        INCOMING FLIGHT:
          15/06/2022, 17:20:00 -> 15/06/2022, 18:35:00 ( 1:15:00 )
          Flight costs 10€ with 0 stops.
    --> Student 3 departing from wien:
        OUTGOING FLIGHT:
          01/06/2022, 06:50:00 -> 01/06/2022, 08:15:00 ( 1:25:00 )
          Flight costs 16€ with 0 stops.
        INCOMING FLIGHT:
          15/